In [ ]:
# For Google Colab use
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd '/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder'    
except ModuleNotFoundError:
    pass

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder


In [ ]:
# Imports
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from numpy.random import seed

import tensorflow as tf

import keras
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import *
from keras.applications import *
from keras import metrics
from keras.losses import binary_crossentropy
from keras import backend as K

try:
    import face_recognition
except ModuleNotFoundError:
    !pip install face_recognition

from Util import pipeline
from Baseline.classifiers import *

     |████████████████████████████████| 100.2MB 31kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=be69e0985a8dcb696934536e537dc933617bae700cc0c2d1759a1986d5955419
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
# General model settings
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 256, 256, 3
EPOCHS = 100
DATA_GENERATOR_SEED = 1337
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2

tf.random.set_seed(DATA_GENERATOR_SEED)
seed(DATA_GENERATOR_SEED)

# Pick dataset; DF_TYPE={'rnd', 'avg'}
for DF_TYPE in ['diff', 'avg', 'rnd']:
    TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30-test'

    TRAIN_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, horizontal_flip = True, fill_mode='nearest', validation_split = VALIDATION_SPLIT)
    TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                        batch_size = BATCH_SIZE,
                                                        class_mode = 'binary', 
                                                        target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                        subset = 'training',
                                                        seed = DATA_GENERATOR_SEED,
                                                        follow_links = True)

    VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, validation_split = VALIDATION_SPLIT)
    VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                             batch_size = BATCH_SIZE,
                                                             class_mode = 'binary', 
                                                             target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                             subset = 'validation',
                                                             seed = DATA_GENERATOR_SEED,
                                                             follow_links = True)

    # Train MesoInception4 w/ F2F weights
    TEST_MODEL = MesoInception4().model
    TEST_MODEL.compile(optimizer = TEST_MODEL.optimizer,
                       loss = TEST_MODEL.loss,
                       metrics = TEST_MODEL.metrics + [metrics.BinaryAccuracy(name = 'acc'),
                                                       metrics.AUC(name = 'auc'),
                                                       metrics.FalsePositives(name = 'fp')])
    
    WEIGHTS_PATH = './Baseline/weights/MesoInception_F2F.h5'
    TEST_MODEL.load_weights(WEIGHTS_PATH) 

    # Make last layer trainable
    for layer in TEST_MODEL.layers:
        layer.trainable = False
    TEST_MODEL.layers[-1].trainable = True

    train_gen_list = list(TRAIN_GENERATOR.classes)
    val_gen_list = list(VALIDATION_GENERATOR.classes)

    train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
    val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)

    pos = train_pos + val_pos
    neg = train_neg + val_neg
    total = pos + neg

    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0

    CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}

    EARLY_STOP = EarlyStopping(monitor='val_auc',
                            patience=EPOCHS//20,
                            mode='max',
                            verbose=1,
                            restore_best_weights=True)

    HISTORY = TEST_MODEL.fit(TRAIN_GENERATOR,
                            steps_per_epoch = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size,
                            validation_data = VALIDATION_GENERATOR,
                            validation_steps = VALIDATION_GENERATOR.n//VALIDATION_GENERATOR.batch_size,
                            epochs = EPOCHS,
                            verbose = 1,
                            class_weight = CLASS_WEIGHT,
                            callbacks = [EARLY_STOP])

    weight_name = WEIGHTS_PATH.split('_')[-1].replace('.h5', '')
    pipeline.evaluate_model(TEST_MODEL = TEST_MODEL,
                            EXPERIMENT_NAME = f'Minimally Trained MesoInception4 on {DF_TYPE} with {weight_name}',
                            TEST_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30-test',
                            WEIGHTS_PATH = '',
                            HISTORY = HISTORY)

Found 15711 images belonging to 2 classes.
Found 3927 images belonging to 2 classes.
Epoch 1/100
122/122 [==============================] - 5227s 43s/step - loss: 0.3311 - acc: 0.4861 - auc: 0.4884 - fp: 302.5366 - val_loss: 0.1020 - val_acc: 0.8943 - val_auc: 0.7332 - val_fp: 222.0000
Epoch 2/100
122/122 [==============================] - 76s 622ms/step - loss: 0.2005 - acc: 0.7124 - auc: 0.7604 - fp: 207.5366 - val_loss: 0.2820 - val_acc: 0.5810 - val_auc: 0.7603 - val_fp: 65.0000
Epoch 3/100
122/122 [==============================] - 76s 618ms/step - loss: 0.1638 - acc: 0.7781 - auc: 0.8407 - fp: 164.2602 - val_loss: 0.4816 - val_acc: 0.3753 - val_auc: 0.6886 - val_fp: 35.0000
Epoch 4/100
122/122 [==============================] - 75s 616ms/step - loss: 0.1528 - acc: 0.7652 - auc: 0.8633 - fp: 141.3496 - val_loss: 0.2278 - val_acc: 0.6625 - val_auc: 0.8083 - val_fp: 70.0000
Epoch 5/100
122/122 [==============================] - 76s 618ms/step - loss: 0.1298 - acc: 0.8169 - auc: 0.89

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 1280 images belonging to 2 classes.
Found 3927 images belonging to 2 classes.
Epoch 1/100
10/10 [==============================] - 1461s 157s/step - loss: 0.5064 - acc: 0.5818 - auc: 0.0000e+00 - fp: 249.4545 - val_loss: 0.7255 - val_acc: 0.0854 - val_auc: 0.4309 - val_fp: 2.0000
Epoch 2/100
10/10 [==============================] - 19s 2s/step - loss: 0.0098 - acc: 0.9993 - auc: 0.0000e+00 - fp: 0.6364 - val_loss: 0.8783 - val_acc: 0.0805 - val_auc: 0.4411 - val_fp: 0.0000e+00
Epoch 3/100
10/10 [==============================] - 19s 2s/step - loss: 0.0042 - acc: 0.9976 - auc: 0.0000e+00 - fp: 1.2727 - val_loss: 0.9019 - val_acc: 0.0810 - val_auc: 0.4485 - val_fp: 0.0000e+00
Epoch 4/100
10/10 [==============================] - 19s 2s/step - loss: 0.0010 - acc: 1.0000 - auc: 0.0000e+00 - fp: 0.0000e+00 - val_loss: 0.9082 - val_acc: 0.0815 - val_auc: 0.4515 - val_fp: 0.0000e+00
Epoch 5/100
10/10 [==============================] - 19s 2s/step - loss: 0.0020 - acc: 1.0000 - auc: 0.000